### Notes on current discoveries:

- All files in storage claim are accessed with the base address '/nfs/'
- Target files are HTML wrapped in .gz, use gzip to unzip
- The inner text of <doc> is stored as the None tag where tag.string != '\n' (is more than just a newline character
- .string of <doc>'s inner text parses inner <a> tags etc. into just their inner text

### Ideas for language model
- NLTK
- BERT

- Load data set -> preprocessing


## Installing Packages

In [ ]:
# # !pip3 install spacy-langdetect
# # !pip3 install language-detector
# # !pip3 install symspellpy
# !pip3 install beautifulsoup4 lxml  # html/xml parser
# !pip3 install torch torchvision
# !pip3 install neuralcoref
# !pip3 install transformers
# !pip3 install sentence-transformers

# #!pip3 install -U spacy   # preprocessing
# #!python -m spacy download en_core_web_sm  # spacy model
# #!pip3 install numpy==1.18

# # !pip3 install spacy==2.1.3
# # !pip3 install transformers==2.2.2
# # !pip3 install neuralcoref

# #!pip3 install tensorflow --upgrade  # for extractive summerizar
# !pip3 install tensorflow
# #!pip3 install bert-extractive-summarizer

#!python -m spacy download en_core_web_md
print("\nsuccessfully installed packages")

     |████████████████████████████████| 122kB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 14.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 296kB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 10.0MB 5.7MB/s ta 0:00:01
     |████████████████████████████████| 6.7MB 84.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 47.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 51.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 46.2MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 58.1MB/s eta 0:00:01
     |████████████████████████████████| 133kB 77.4MB/s eta 0:00:01
     |████████████████████████████████| 194kB 84.3MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 82.6MB

  Created wheel for srsly: filename=srsly-1.0.3-cp36-cp36m-linux_x86_64.whl size=575383 sha256=30e43638decd72e7928e009aea0ef786dde6c2a7b9ad76523d176225eed833a8
  Stored in directory: /root/.cache/pip/wheels/b9/06/ad/f25782fed6621d8b55801f8afe94ed631f2a86f117b2b6374f
Successfully built cymem srsly
ERROR: botocore 1.19.10 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.25.3 which is incompatible.
  Found existing installation: tqdm 4.35.0
    Uninstalling tqdm-4.35.0:
      Successfully uninstalled tqdm-4.35.0
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 1.3MB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 16.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 42.5MB/s eta 0:00:01
     |████████████████████████████████| 675kB 40.8MB/s eta 0:00:01
     |████████████████████████████████| 890kB 49.8MB/s eta 0:00:01
     |█████████████████████████████

## Loading Data Set

In [1]:
## IMPORT DEPENDENCIES

from bs4 import BeautifulSoup as bs
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import os
from collections import OrderedDict
print ("loading data set dependencies successful")

loading data set dependencies successful


In [7]:
## SET FILE META VARIABLES

corpus_path = "/nfs/trects-kba2014-filtered" # directory of corpus of gzipped html files
topics_path = corpus_path + "/test-topics.xml"
doc_tags = ['topic_id','streamid', 'docid', 'yyyymmddhh', 'kbastream', 'zulu', 'epoch', 'title', 'text', 'url'] # doc fields
topic_tags = ['id', 'title', 'description', 'start','end','query','type'] # topic fields
test_file_addr = corpus_path + "/1/2012-02-22-15.gz"

In [3]:
# open and get beautifulsoup object from markup file
def open_markup_file(addr, gz=True, xml=False, verbose=False):
    markup = None
    f = None
    
    if verbose:
        print(addr)

    if gz:
        f = gzip.open(addr)
    else:
        f = open(addr)
        
    if xml == False:
        markup = bs(f)  # open as html
    else:
        markup = bs(f, "xml")
        
    f.close()
    return markup


# parse markup and return 2D list [entry:tags]
def parse_markup(markup, entry_list, find_tag="doc", tag_list=doc_tags, topic_id=None):
    for e in markup.find_all(find_tag):
        entry = OrderedDict.fromkeys(tag_list)
        if topic_id is not None:
            entry['topic_id'] = topic_id
        for c in e.children:  # children use direct children, descendants uses all
            if c.name in entry:
                entry[c.name] = c.string
            elif c.name is None and c.string != '\n':  # inner body of <doc> tag
                entry['text'] = c.string
#             elif c.name is not None:
#                 print("Entry has unexpected field: " + str(c.name))
#                 print("parent field: " + c.parent.name)
#                 print("field content: " + str(c))
        
#         missing_fields = [k for (k,v) in entry.items() if v == None]
#         for m in missing_fields:
#             print("Entry is missing field: " + m)
#             try:
#                 for (k,v) in entry.items():
#                     print(k,v)
#             except:
#                 print("could not print entry dictionary")
        entry_list.append(list(entry.values()))
        
            
# recursively find gz html files from a directory address
def search_dir(path):    
    # separate the subdirectories and html files 
    # (help maintain sequential order of insertion)
#     subdirs = []
    gz_paths = []
    for f in os.scandir(path):
#         if f.is_dir():
#             subdirs.append(f.path)
        if os.path.splitext(f.path)[-1].lower() == ".gz":
            gz_paths.append(f.path)
    
#     # search subdirs
#     for sd in subdirs:
#         search_dir(sd, gz_paths)
    return gz_paths


def list_to_dataframe(markup_list, tags):
    return pd.DataFrame(markup_list, columns=tags)

In [4]:
# load topics into dataframe
def load_topics(path):
    topics_list = []
    
    parse_markup(open_markup_file(path, gz=False, xml=True), 
                    topics_list, find_tag="event", tag_list=topic_tags)
    
    
    return  list_to_dataframe(topics_list, topic_tags)

topics = load_topics(topics_path)

In [7]:
print("Topics loaded successfuly")
print(topics.head(4))

Topics loaded successfuly
  id                                title  \
0  1      2012 Buenos Aires Rail Disaster   
1  2  2012 Pakistan garment factory fires   
2  3                 2012 Aurora shooting   
3  4       Wisconsin Sikh temple shooting   

                                         description       start         end  \
0  http://en.wikipedia.org/wiki/2012_Buenos_Aires...  1329910380  1330774380   
1  http://en.wikipedia.org/wiki/2012_Pakistan_gar...  1347368400  1348232400   
2  http://en.wikipedia.org/wiki/2012_Aurora_shooting  1342766280  1343630280   
3  http://en.wikipedia.org/wiki/Wisconsin_Sikh_te...  1344180300  1345044300   

                      query      type  
0  buenos aires train crash  accident  
1     pakistan factory fire  accident  
2         colorado shooting  shooting  
3      sikh temple shooting  shooting  


In [30]:
# load all formatted gzipped html files into dataframe
def load_corpus(path):
    corpus_list = []
    gz_paths = []
    for topic_id in topics['id'].to_numpy():
        id_path = corpus_path + "/" + topic_id + "/"  # every topic id correlates to subfolder named after it
        gz_paths = search_dir(id_path)
    for gz_path in tqdm(gz_paths, position=0, leave=True):
        parse_markup(open_markup_file(gz_path, verbose=False),
                        corpus_list, topic_id=topic_id)
#     for topic_id in topics['id'].to_numpy():
#         id_path = corpus_path + "/" + topic_id + "/"  # every topic id correlates to subfolder named after it
#         gz_paths = search_dir(id_path)
#         for gz_path in gz_paths:
#             parse_markup(open_markup_file(gz_path, verbose=True),
#                             corpus_list, topic_id=topic_id)
    return list_to_dataframe(corpus_list, doc_tags)

corpus = load_corpus(corpus_path)
#print("Corpus loaded Successfully")

 99%|█████████▉| 239/241 [01:24<00:00,  4.81it/s]














100%|██████████| 241/241 [01:24<00:00,  2.86it/s]


In [21]:
print("Corpus loaded succesfully: " + str(len(corpus)) + " documents loaded.")
print(corpus.head(4))

Corpus loaded succesfully: 1578 documents loaded.
  topic_id                                     streamid  \
0       10  1354113657-a4417f055ea5ae84207a4edb4dad881b   
1       10  1354112039-110cc86ea7a8a1b58306dfade5b300ec   
2       10  1354114192-a4417f055ea5ae84207a4edb4dad881b   
3       10  1354114426-6c8d58d994c0e3243ee8dca8f34516a4   

                              docid     yyyymmddhh        kbastream  \
0  a4417f055ea5ae84207a4edb4dad881b  2012-11-28-14  MAINSTREAM_NEWS   
1  110cc86ea7a8a1b58306dfade5b300ec  2012-11-28-14  MAINSTREAM_NEWS   
2  a4417f055ea5ae84207a4edb4dad881b  2012-11-28-14  MAINSTREAM_NEWS   
3  6c8d58d994c0e3243ee8dca8f34516a4  2012-11-28-14           WEBLOG   

                     zulu       epoch  \
0  2012-11-28T14:40:57.0Z  1354113657   
1  2012-11-28T14:13:59.0Z  1354112039   
2  2012-11-28T14:49:52.0Z  1354114192   
3  2012-11-28T14:53:46.0Z  1354114426   

                                               title  \
0  Morning Briefing: Support grows f

In [8]:
test_file_df = list_to_dataframe(parse_markup(open_markup_file(test_file_addr)), doc_tags)

TypeError: parse_markup() missing 1 required positional argument: 'entry_list'

## Preprocessing

### Preprocessing Notes

####Links
-  https://stackoverflow.com/questions/54938815/data-preprocessing-for-nlp-pre-training-models-e-g-elmo-bert
- https://www.activestate.com/blog/natural-language-processing-nltk-vs-spacy/

####Notes
- Thinking of using spacy as it provides faster simpler preprocessing options


In [49]:
## IMPORT DEPENDENCIES

import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 

import spacy

#from stop_words import get_stop_words
##from nltk.stem.porter import PorterStemmer
import re
#import nltk
#from nltk.tokenize import word_tokenize
#from language_detector import detect_language

#import pkg_resources
#from symspellpy import SymSpell, Verbosity

print("preprocessing dependencies import successful")

preprocessing dependencies import successful


In [51]:
nlp = spacy.load("en_core_web_sm")  # en_core_web_lg nearly 80x larger, marginal differences

# was in TaD solution
# nlp.remove_pipe('tagger')
# nlp.remove_pipe('parser')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [34]:
# ## LOAD SPELL CHECKER

# sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
# dictionary_path = pkg_resources.resource_filename(
#     "symspellpy", "frequency_dictionary_en_82_765.txt")
# if sym_spell.word_count:
#     pass
# else:
#     sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

## SENTENCE LEVEL PREPROCESSING

In [39]:
# #@Tokenize
# def spacy_tokenize(string):
#   tokens = list()
#   doc = nlp(string)
#   for token in doc:
#     tokens.append(token)
#   return tokens

# #@Normalize
# def normalize(tokens):
#   normalized_tokens = list()
#   for token in tokens:
#     normalized = token.text.lower().strip()
#     if ((token.is_alpha or token.is_digit)):
#       normalized_tokens.append(normalized)
#   return normalized_tokens
#   return normalized_tokens

# #@Tokenize and normalize
# def tokenize_normalize(string):
#   return normalize(spacy_tokenize(string))

In [41]:
# print(tokenize_normalize("this string right here is an example, I'm just testing what spacy does"))

NameError: name 'nlp' is not defined

## Vectorization

In [ ]:
from sentence_transformers import SentenceTransformer

I1105 14:59:37.771246 139884767500096 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
I1105 14:59:37.786265 139884767500096 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [1]:
sent_model = SentenceTransformer('distilbert-base-nli-mean-tokens')

NameError: name 'SentenceTransformer' is not defined

## Summarization
### bert-extractive-summarizer is causing import/dependecy issues

In [ ]:
# from summarizer import Summarizer

I1104 13:33:23.269840 139647819040576 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
I1104 13:33:23.284047 139647819040576 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [5]:
# sum_model = Summarizer()
# print("loaded model")
# test_sum_text = corpus['text'].iloc(0)
# test_sum = sum_model(test_sum_text, min_length=10)
# print("".join(test_sum))

I1103 15:40:42.116671 139697169651520 file_utils.py:331] https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp0iglg5_c


I1103 15:40:42.539464 139697169651520 file_utils.py:346] copying /tmp/tmp0iglg5_c to cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
I1103 15:40:42.541013 139697169651520 file_utils.py:350] creating metadata file for /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
I1103 15:40:42.542294 139697169651520 file_utils.py:359] removing temp file /tmp/tmp0iglg5_c
I1103 15:40:42.542935 139697169651520 configuration_utils.py:160] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
I1103 15:40:42.543757 139697169651520 configuration_u

I1103 15:40:42.914231 139697169651520 file_utils.py:331] https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-pytorch_model.bin not found in cache or force_download set to True, downloading to /tmp/tmprbt3v5_x


KeyboardInterrupt: 

In [4]:
# import tensorflow as tf
# tf.version.VERSION
# !python --version

Python 3.6.8
